# Bcubed precision and recall

* Definition in *Amigó E, Gonzalo J, Artiles J et alA comparison of extrinsic clustering evaluation metrics based on formal constraints.   Information Retrieval. 12. 461-486. 10.1007/s10791-008-9066-8.*

![](BCPR.png)


1. On the web you can quickly find definitions of bcubed precision and recall: see eg <https://www.researchgate.net/figure/Example-of-computing-the-BCubed-precision-and-recall-for-one-item_fig10_225548032> for a nice picture.
2. Your function  will have as input two vectors: true and predicted. 
3. The sum of the values in that list equals the numbefr of pages of the PDF file.
4. The  length of that list is the number of documents in the PDF file
5. The numbers in the list indicate the lengths of the documents. So L[i] gives the length in number of pages of the i-th document in the PDF file.
6. You find an example on `DocumentSplitting/data/WobGelderland/Doclengths_of_the_individual_docs.json` for the files in `/DocumentSplitting/data/WobGelderland/ConcatenatedPDFS`  
 

### baselines

* every page starts a new doc ([1,1,1,1,......])
* there is only one doc  ([X]) for X the length of the compolete document
* but also more informed ones, like the mode and the median of the true splits 
 

In [2]:
import json
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
from collections import defaultdict
%matplotlib inline


In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from networks.vgg_lstm import Vgg_Lstm
# from networks.vgg_bert_custom import vgg_bert

In [3]:
import re
import nltk
import pickle
# import pypdfium2
# import pytesseract
import numpy as np
import pandas as pd

from PIL import Image
from bs4 import BeautifulSoup
from gensim.models import Doc2Vec
from stop_words import get_stop_words
from gensim.models.doc2vec import TaggedDocument
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer

In [8]:
feature_length = 1024
model_dbow = Doc2Vec.load("gensim/pdf_split_d2v_gensim_{}_db.mod".format(feature_length))
model_dmm = Doc2Vec.load("gensim/pdf_split_d2v_gensim_{}_dm.mod".format(feature_length))
vectorizer_gensim = ConcatenatedDoc2Vec([model_dbow, model_dmm])
model_lstm = Vgg_Lstm(img_dim=3, embedding_dim=2048, output_dim=2)
# model_bert = vgg_bert(in_ch=3, out_ch=2)
with open('models/logregmodel_lstmlogreg_model_1024_dbow_dm_concate_100.sav', 'rb') as m:
    model_log = pickle.load(m)

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

def tag_page(prediction):
    """
    :param prediction: classify prediction array: e.g [1, 0, 0, 1, 0, 0, 1, 0, 1]
    :return: tag page: e.g. [3, 3, 2, 1]
    """
    tag = np.split(prediction, np.argwhere(prediction == 1).flatten())
    tag = [len(tag[i]) for i in range(len(tag)) if len(tag[i])]
    tag = np.array(tag)

    return tag

def cleanText(text):
    text = BeautifulSoup(text, "html.parser").text
    text = re.sub(r'\|\|\|', r' ', text)
    text = re.sub(r'\\n', r' ', text)
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text


/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [9]:
df = pd.read_csv('../sample_data/sample_f.csv')
df['name'].unique()[0]

FileNotFoundError: [Errno 2] No such file or directory: '../sample_data/sample_f.csv'

In [10]:
def data_loader(df):
    df.fillna('', inplace=True)
    df['text_processed'] = df['text'].apply(cleanText)
    path = df['fname'].tolist()
    text = df['text_processed'].tolist()
    # label = df['labels'].tolist()
    return path, text

In [11]:
class Prediction():
    def __init__(self, path, text, data_pct="25", vectorizer_type='gensim', model_type='log'):
        self.vectorizer_type = vectorizer_type
        self.model_type = model_type
        self.path = path
        self.text = text
        self.data_percentage = data_pct
        self.output = []

    def run_batch(self):
        result = []
        for i in range(len(self.path)):
            fname = self.path[i]
            image = cv2.imread(fname)
            text = self.text[i]
            predict = self.run_single(image, text)
            result.append(predict)
        result = np.array(result)
        result = tag_page(result)
        return result


    def run_single(self, image, text):
        if self.model_type == 'log':
            text = tokenize_text(text)
            gensim_vectors = vectorizer_gensim.infer_vector(text)
            prediction = model_log.predict([gensim_vectors])
            return int(prediction.item())
        elif self.model_type == 'vgg_lstm':
            text = tokenize_text(text)
            gensim_vectors = vectorizer_gensim.infer_vector(text)
            ...
            return None

In [23]:
path_to_csv = '../sample_data/sample_f.csv'
whole_df = pd.read_csv(path_to_csv)
df_list = [whole_df[whole_df['name']==x] for x in list(whole_df['name'].unique())]

rs = []
for df in df_list:
    path, text = data_loader(df)
    model_predict = Prediction(path=path, text=text)
    rs.append(list(model_predict.run_batch()))

/var/folders/k7/f9jwgyf91j53cx39tj4fmv700000gp/T/ipykernel_24551/907674630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna('', inplace=True)
/var/folders/k7/f9jwgyf91j53cx39tj4fmv700000gp/T/ipykernel_24551/907674630.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_processed'] = df['text'].apply(cleanText)
/var/folders/k7/f9jwgyf91j53cx39tj4fmv700000gp/T/ipykernel_24551/907674630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

In [ ]:
# examples
# vb_truth, vb_pred = [1,2,1,3,3,5], [1,1,2,3,4,4]

In [ ]:
def make_index(split):
    '''Turns a doc length vector like [1,2,1,3,3,5] into a dict with pagenumbers as keys and the set of all 
    pagenumbers in the same document as value.
    This thus is an index which gives for every page its cluster.'''
    l= sum(split)
    pages= list(np.arange(l))
    out = defaultdict(set)
    for block_length in split:
        block= pages[:block_length]
        pages= pages[block_length:]
        for page in block:
            out[page]= set(block)
    return out

#test
make_index(vb_truth)
#tests

In [ ]:
def Bcubed(truth,pred):
    assert sum(truth)==sum(pred)  # same amount of pages
    truth,pred = make_index(truth), make_index(pred)
    
    df  ={i:{'size':len(truth[i]),'P':0,'R':0,'F1':0} for i in truth}
    for i in truth:
        df[i]['P']= len(truth[i] & pred[i])/len(pred[i]) 
        df[i]['R']= len(truth[i] & pred[i])/len(truth[i])
        df[i]['F1']= (2*df[i]['P']*df[i]['R'])/(df[i]['P']+df[i]['R'])
    df= pd.DataFrame.from_dict(df, orient='index')
    df.index_name='PageNr'
    return  df


def MeanBcubed(truth,pred):
    assert sum(truth)==sum(pred)  # same amount of pages
    return Bcubed(truth,pred).mean()


print(MeanBcubed(vb_truth,vb_pred))

Bcubed(vb_truth,vb_pred)


# On corpus

* baseline is fixed cluster/document size with all the median pagelength

In [ ]:
f= open('../corpus1/TrainTestSet/Trainset/Doclengths_of_the_individual_docs_TRAIN.json')
truth_corpus=json.load(f)

In [ ]:
def fixedpage(truth,docsize=3):
    number_of_blocks= sum(truth)//docsize
    rest = sum(truth) % docsize
    if rest !=0:
        return [docsize for _ in range(number_of_blocks)]+[rest]
    else:
        return [docsize for _ in range(number_of_blocks)]  
    
D ={pdf: MeanBcubed(truth_corpus[pdf], fixedpage(truth_corpus[pdf],6))
   for pdf in truth_corpus}
results= pd.DataFrame.from_dict(D,orient='index')
print(results.describe())
sns.boxplot(data=results[['P','R','F1']]);    

In [ ]:
title='Mean Bcubed P,R,F1 for each document in the corpus1 Train set with fixed median (6) doc. size.'
results[['P','R','F1']].sort_values('R').reset_index().plot( title=title,
                                               figsize=(20,8));

In [ ]:
results.plot.scatter(x='R', y='P', figsize=(10,8));